# FRP SSH Tunnel - Google Colab Example

This notebook demonstrates how to set up SSH tunneling from Google Colab to your local machine using FRP (Fast Reverse Proxy).

## Prerequisites

1. A server (GCP/VPS) running the FRP server
2. Your server's IP address
3. Your FRP authentication token

## Quick Setup

In [ ]:
# Configuration - Update these values
SERVER_IP = "YOUR_SERVER_IP_HERE"  # e.g., "35.220.164.135"
TOKEN = "YOUR_TOKEN_HERE"          # e.g., "frp_a1b2c3d4e5f6..."
USERNAME = "colab"                 # SSH username
PORT = 6001                        # SSH port (6001-6010)

print(f"Configuration:")
print(f"Server: {SERVER_IP}")
print(f"Port: {PORT}")
print(f"Username: {USERNAME}")
print(f"Token: {'*' * 20}...")

## Step 1: Download and Run Setup Script

In [ ]:
# Download the FRP client script
!wget -q https://raw.githubusercontent.com/your-username/frp-ssh-tunnel/main/scripts/frp-client-colab.sh
!chmod +x frp-client-colab.sh
print("✅ Script downloaded successfully")

In [ ]:
# Run the setup script
!bash frp-client-colab.sh {USERNAME} {PORT} {SERVER_IP} {TOKEN}

## Step 2: Verify Connection

In [ ]:
# Check if FRP client is running
!ps aux | grep frpc | grep -v grep

In [ ]:
# Check SSH service status
!sudo systemctl status ssh --no-pager

In [ ]:
# Test local SSH connection
!ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null localhost 'echo "SSH is working!"'

## Step 3: Connection Information

In [ ]:
# Display connection information
print("🎉 Setup Complete!")
print("="*50)
print(f"SSH Command: ssh -p {PORT} {USERNAME}@{SERVER_IP}")
print(f"SCP Upload: scp -P {PORT} file.txt {USERNAME}@{SERVER_IP}:~/")
print(f"SCP Download: scp -P {PORT} {USERNAME}@{SERVER_IP}:~/file.txt ./")
print("="*50)
print("💡 Tip: Add this to your ~/.ssh/config for easier access:")
print(f"""Host colab-{PORT}
    HostName {SERVER_IP}
    User {USERNAME}
    Port {PORT}
    IdentityFile ~/.ssh/id_rsa""")

## Step 4: File Transfer Examples

In [ ]:
# Create a sample file to transfer
import os
import datetime

# Create sample data
sample_data = f"""Hello from Google Colab!
Generated at: {datetime.datetime.now()}
Runtime ID: {os.environ.get('COLAB_GPU', 'CPU')}
Python version: {os.sys.version}
"""

with open('colab-sample.txt', 'w') as f:
    f.write(sample_data)

print("📄 Sample file created: colab-sample.txt")
print("Contents:")
!cat colab-sample.txt

## Step 5: Keep Connection Alive

In [ ]:
# Keep Colab session alive (optional)
import time
import threading

def keep_alive():
    """Keep Colab session alive by printing status every 5 minutes"""
    count = 0
    while True:
        time.sleep(300)  # 5 minutes
        count += 1
        print(f"🔄 Keep-alive #{count} - {datetime.datetime.now()}")
        
        # Check if FRP client is still running
        result = os.system('pgrep frpc > /dev/null')
        if result == 0:
            print("✅ FRP client is running")
        else:
            print("❌ FRP client stopped - restarting...")
            os.system(f'bash frp-client-colab.sh {USERNAME} {PORT} {SERVER_IP} {TOKEN}')

# Start keep-alive thread
keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

print("🔄 Keep-alive thread started")
print("💡 This will keep your Colab session and SSH tunnel active")

## Troubleshooting

In [ ]:
# Troubleshooting commands
print("🔍 Troubleshooting Information")
print("="*40)

print("\n📊 System Info:")
!uname -a

print("\n🌐 Network Info:")
!ip addr show | grep inet

print("\n🔧 FRP Process:")
!ps aux | grep frp | grep -v grep || echo "No FRP processes found"

print("\n📝 FRP Logs (last 10 lines):")
!tail -10 ~/logs/frpc.log 2>/dev/null || echo "No logs found"

print("\n🔑 SSH Keys:")
!ls -la ~/.ssh/ 2>/dev/null || echo "No SSH directory found"

## Manual Restart (if needed)

In [ ]:
# Manual restart of FRP client
print("🔄 Restarting FRP client...")

# Stop existing client
!pkill frpc || echo "No existing client to stop"

# Wait a moment
import time
time.sleep(2)

# Restart client
!bash frp-client-colab.sh {USERNAME} {PORT} {SERVER_IP} {TOKEN}

print("✅ Restart complete")